<a href="https://colab.research.google.com/github/kunalrajsingh/NLP_AQG_PRO9JECT/blob/main/Automatic_QUES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
model_name = "valhalla/t5-small-qg-prepend"  # A T5 model fine-tuned for question generation
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [3]:
def generate_questions(passage):
    # Prepare the input text in a format T5 expects
    input_text = f"generate question: {passage}"

    # Tokenize the input
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Generate questions
    outputs = model.generate(inputs, max_length=64, num_return_sequences=3, num_beams=5)

    # Decode the generated questions
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Combine all questions into a single string for easy display
    return "\n".join(questions)


In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
model_name = "valhalla/t5-small-qg-prepend"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [5]:
def generate_diverse_questions(passage):
    # Template prompts for different question types
    question_types = [
        "Who question: ",
        "What question: ",
        "When question: ",
        "Why question: ",
        "How question: "
    ]

    # Store generated questions in a list
    generated_questions = []

    # Generate a question for each template
    for q_type in question_types:
        # Prepare input text by combining template with passage
        input_text = f"{q_type}{passage}"

        # Tokenize and encode the input
        inputs = tokenizer.encode(input_text, return_tensors="pt")

        # Generate question with specific settings for diversity
        outputs = model.generate(
            inputs,
            max_length=64,
            num_return_sequences=1,
            num_beams=3,  # Using fewer beams for speed and diversity
            early_stopping=True
        )

        # Decode the generated question and add it to the list
        question = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_questions.append(question)

    # Combine all questions into a single string for easy display
    return "\n".join(generated_questions)


In [6]:
import gradio as gr

# Set up the Gradio interface
interface = gr.Interface(
    fn=generate_diverse_questions,  # The function that generates diverse questions
    inputs=gr.Textbox(label="Passage", placeholder="Enter the passage text here"),
    outputs=gr.Textbox(label="Generated Questions"),
    title="Diverse Question Generator",
    description="Enter a passage to generate diverse questions based on its content."
)

# Launch the Gradio interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a5ae4ba06c48e21e3f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
